In [1]:
from general_vxm_utils import *

full_train_file = '/home/vib9/src/SL-Net/jupyter/partitions/train.txt'
full_val_file = '/home/vib9/src/SL-Net/jupyter/partitions/val.txt'
scarce_train_file = '/home/vib9/src/SL-Net/jupyter/partitions/scarce_train.txt'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

target_label_numbers = [2.0, 3.0, 16.0, 41.0, 42.0]

In [2]:
"""
Setup vanilla UNet
"""

enc_nf = [128, 128, 128, 128]
dec_nf = [128, 128, 128, 128]

model1 = UNet(input_ch=1, out_ch=6, enc_nf=enc_nf, dec_nf=dec_nf)

In [ ]:
"""
Train the vanilla UNet just to make sure that we are alright with our model and data
"""

train_soft1, val_scores1, train_vars1, val_vars1 = train_net(model1,
                                                            1,
                                                            32,
                                                            1e-4,
                                                            target_label_numbers,
                                                            train_path=full_train_file,
                                                            val_path=full_val_file)

save_results(model1, train_soft1, val_scores1, train_vars1, val_vars1, "default-train")

INFO: Creating dataset with 5865 examples
INFO: Creating dataset with 732 examples
Validation round: 100%|█████████▉| 731/732 [00:07<00:00, 113.59batch/s]
                                                                       INFO: Validation Dice Loss: 0.8863792723965369
Validation round:  70%|██████▉   | 511/732 [00:05<00:01, 115.72batch/s]


Validation round:  99%|█████████▉| 726/732 [00:06<00:00, 125.18batch/s]
                                                                       INFO: Validation Dice Loss: 0.18851323264707573
Validation round: 100%|██████████| 732/732 [00:07<00:00, 105.08batch/s]
                                                                       INFO: Validation Dice Loss: 0.12621685726468232
Validation round:  30%|███       | 222/732 [00:03<00:08, 62.79batch/s]


Validation round:  99%|█████████▉| 725/732 [00:08<00:00, 128.19batch/s]
                                                                       INFO: Validation Dice Loss: 0.11606292233643101
Validation round:  89%|████████▉ | 651/732 [00:07<00:00, 116.35batch/s]


Validation round:  98%|█████████▊| 718/732 [00:08<00:00, 124.00batch/s]
                                                                       INFO: Validation Dice Loss: 0.1114178824668947
Epoch 1/1:  99%|█████████▉| 5792/5865 [01:42<00:06, 11.57img/s, loss (batch)=0.0614]

In [ ]:
"""
Setup SLN UNet
"""

model2 = SLN_UNet(input_ch=1, out_ch=6, superblock_size=256, depth=4, W=None, b=None, train_block=True)

In [ ]:
"""
Train the vanilla UNet just to make sure that we are alright with our model and data
"""
train_soft2, val_scores2, train_vars2, val_vars2 = train_net(model2,
                                                        1,
                                                        32,
                                                        1e-4,
                                                        target_label_numbers,
                                                        train_path=full_train_file,
                                                        val_path=full_val_file)

save_results(model2, train_soft2, val_scores2, train_vars2, val_vars2, "sln")

np.save("/home/vib9/src/SL-Net/superlayer/models/superblocks/sln_W", model2.W.cpu().data)
print("Saved weight to: /home/vib9/src/SL-Net/superlayer/models/superblocks/sln_W.npy!")
np.save("/home/vib9/src/SL-Net/superlayer/models/superblocks/sln_b", model2.b.cpu().data)
print("Saved weight to: /home/vib9/src/SL-Net/superlayer/models/superblocks/sln_b.npy!")

In [ ]:
plot_subplot_array("default-train", 
                   "sln", 
                   [0, 1.1], 
                   ["Default", "sln"])

In [ ]:
"""
Setup SLN UNet
"""
weight3 = np.load("/home/vib9/src/SL-Net/superlayer/models/superblocks/sln_W.npy")
bias3 = np.load("/home/vib9/src/SL-Net/superlayer/models/superblocks/sln_b.npy")

model3 = SLN_UNet(input_ch=1, out_ch=6, superblock_size=256, depth=4, W=weight3, b=bias3)

In [ ]:
train_soft3, val_scores3, train_vars3, val_vars3 = train_net(model3,
                                                        1,
                                                        32,
                                                        1e-4,
                                                        target_label_numbers,
                                                        train_path=full_train_file,
                                                        val_path=full_val_file)

In [ ]:
model3.state_dict()